### Team: David Flores, Mauricio García, Elena Zamudio, David Girma, Francisco García & Nallely González

#### 1. Import the 11 CSV files as DataFrames

In [4]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

In [5]:
# Load paths
diseases_path = Path("Resources/Data/30-70cancerChdEtc_OK.csv")
adolescentBirth_path = Path("Resources/Data/adolescentBirthRate_OK.csv")
airPollutionDeathRate_pulmonary_path = Path("Resources/Data/airPollutionDeathRate_OK_pulmonary.csv")
airPollutionDeathRate_ischeamic_path = Path("Resources/Data/airPollutionDeathRate_OK_Ischaemic_heart_disease.csv")
airPollutionDeathRate_respinfections_path = Path("Resources/Data/airPollutionDeathRate_OK_Lower_respiratory_infections.csv")
airPollutionDeathRate_stroke_path = Path("Resources/Data/airPollutionDeathRate_OK_Stroke.csv")
airPollutionDeathRate_cancers_path = Path("Resources/Data/airPollutionDeathRate_OK_Trachea_bronchus_lung_cancers.csv")
crudeSuicideRates_path = Path("Resources/Data/crudeSuicideRates.csv")
mortalityRatePoisoning_path = Path("Resources/Data/mortalityRatePoisoning_OK.csv")
lifeExpectancyAtBirth_path = Path("Resources/Data/lifeExpectancyAtBirth_OK.csv")
HALElifeExpectancyAtBirth_path = Path("Resources/Data/HALElifeExpectancyAtBirth_OK.csv")

# Read CSV data files and store them into Pandas DataFrames
diseases_df = pd.read_csv(diseases_path)
adolescentBirth_df = pd.read_csv(adolescentBirth_path)
airPollutionDeathRate_pulmonary_df = pd.read_csv(airPollutionDeathRate_pulmonary_path)
airPollutionDeathRate_ischeamic_df = pd.read_csv(airPollutionDeathRate_ischeamic_path)
airPollutionDeathRate_respinfections_df = pd.read_csv(airPollutionDeathRate_respinfections_path)
airPollutionDeathRate_stroke_df = pd.read_csv(airPollutionDeathRate_stroke_path)
airPollutionDeathRate_cancers_df = pd.read_csv(airPollutionDeathRate_cancers_path)
crudeSuicideRates_df = pd.read_csv(crudeSuicideRates_path)
mortalityRatePoisoning_df = pd.read_csv(mortalityRatePoisoning_path)
lifeExpectancyAtBirth_df = pd.read_csv(lifeExpectancyAtBirth_path)
HALElifeExpectancyAtBirth_df = pd.read_csv(HALElifeExpectancyAtBirth_path)

# Display the header of a DataFrame
diseases_df.head()

,Indicator_data,CASE_ID,first_looltip
0,Probability (%) of dying between age 30 and ex...,"Afghanistan,2016,Both sexes",29.8
1,Probability (%) of dying between age 30 and ex...,"Afghanistan,2016,Male",31.8
2,Probability (%) of dying between age 30 and ex...,"Afghanistan,2016,Female",27.7
3,Probability (%) of dying between age 30 and ex...,"Afghanistan,2015,Both sexes",29.8
4,Probability (%) of dying between age 30 and ex...,"Afghanistan,2015,Male",31.9


#### 2. Design a combined DataFrame

In [7]:
# Set CASE_ID as the first column & rename columns
combined_df = diseases_df[["CASE_ID","first_looltip"]]
combined_df.columns = ["CASE_ID","30-70 Death (%)"]

combined_df

,CASE_ID,30-70 Death (%)
0,"Afghanistan,2016,Both sexes",29.8
1,"Afghanistan,2016,Male",31.8
2,"Afghanistan,2016,Female",27.7
3,"Afghanistan,2015,Both sexes",29.8
4,"Afghanistan,2015,Male",31.9
...,...,...
2740,"Zimbabwe,2005,Male",22.1
2741,"Zimbabwe,2005,Female",22.9
2742,"Zimbabwe,2000,Both sexes",21.6
2743,"Zimbabwe,2000,Male",21.6


In [8]:
# adolescentBirth & airPollution DataFrames (2-7) do not have 'gender' included in the CASE_ID.
# Hence adding the other DataFrames then (8 - 11)

# crudeSuicideRates & mortalityRatePoisoning DataFrames (8 & 9) have exact CASE_ID (same sample years: 2000, 2005, 2010, 2015 & 2016).
combined_df = pd.merge(combined_df, crudeSuicideRates_df, left_index=True, right_index=True)
combined_df = combined_df.drop(combined_df.columns[[2, 3]], axis=1)

combined_df = pd.merge(combined_df, mortalityRatePoisoning_df, left_index=True, right_index=True)
combined_df = combined_df.drop(combined_df.columns[[3, 4]], axis=1)
combined_df.columns = ["CASE_ID", "30-70 Death (%)", "Crude Suicide Rate per 100k", "Mortality by Poisoning Rate per 100k"]

combined_df

,CASE_ID,30-70 Death (%),Crude Suicide Rate per 100k,Mortality by Poisoning Rate per 100k
0,"Afghanistan,2016,Both sexes",29.8,0.0,1.19
1,"Afghanistan,2016,Male",31.8,0.0,0.53
2,"Afghanistan,2016,Female",27.7,0.0,1.89
3,"Afghanistan,2015,Both sexes",29.8,4.8,1.22
4,"Afghanistan,2015,Male",31.9,7.8,0.55
...,...,...,...,...
2740,"Zimbabwe,2005,Male",22.1,21.7,2.85
2741,"Zimbabwe,2005,Female",22.9,6.4,2.62
2742,"Zimbabwe,2000,Both sexes",21.6,12.9,3.44
2743,"Zimbabwe,2000,Male",21.6,20.6,3.75


In [9]:
# Adding lifeExpectancyAtBirth DataFrame (10)
# This DataFrame only includes years 2000, 2010 and 2015 (NaN values for the other years where life expectancy is unavailable). 
combined_df = combined_df.merge(lifeExpectancyAtBirth_df, how='outer', left_on='CASE_ID', right_on='case_id')
combined_df = combined_df.drop(combined_df.columns[[4, 5]], axis=1)
combined_df.rename(columns={'First Tooltip':'Life Expectancy'}, inplace=True)

combined_df

,CASE_ID,30-70 Death (%),Crude Suicide Rate per 100k,Mortality by Poisoning Rate per 100k,Life Expectancy
0,"Afghanistan,2016,Both sexes",29.8,0.0,1.19,NaN
1,"Afghanistan,2016,Male",31.8,0.0,0.53,NaN
2,"Afghanistan,2016,Female",27.7,0.0,1.89,NaN
3,"Afghanistan,2015,Both sexes",29.8,4.8,1.22,61.65
4,"Afghanistan,2015,Male",31.9,7.8,0.55,61.04
...,...,...,...,...,...
2740,"Zimbabwe,2005,Male",22.1,21.7,2.85,NaN
2741,"Zimbabwe,2005,Female",22.9,6.4,2.62,NaN
2742,"Zimbabwe,2000,Both sexes",21.6,12.9,3.44,46.57
2743,"Zimbabwe,2000,Male",21.6,20.6,3.75,45.15


In [10]:
# Adding HALElifeExpectancyAtBirth DataFrame (11)
# Same as before, this DataFrame only includes years 2000, 2010 and 2015.
combined_df = combined_df.merge(HALElifeExpectancyAtBirth_df, how='outer', left_on='CASE_ID', right_on='case_id')
combined_df = combined_df.drop(combined_df.columns[[5, 6]], axis=1)
combined_df.rename(columns={'First Tooltip':'Healthy Life Expectancy (HALE)'}, inplace=True)

combined_df

,CASE_ID,30-70 Death (%),Crude Suicide Rate per 100k,Mortality by Poisoning Rate per 100k,Life Expectancy,Healthy Life Expectancy (HALE)
0,"Afghanistan,2016,Both sexes",29.8,0.0,1.19,NaN,NaN
1,"Afghanistan,2016,Male",31.8,0.0,0.53,NaN,NaN
2,"Afghanistan,2016,Female",27.7,0.0,1.89,NaN,NaN
3,"Afghanistan,2015,Both sexes",29.8,4.8,1.22,61.65,52.60
4,"Afghanistan,2015,Male",31.9,7.8,0.55,61.04,52.78
...,...,...,...,...,...,...
2740,"Zimbabwe,2005,Male",22.1,21.7,2.85,NaN,NaN
2741,"Zimbabwe,2005,Female",22.9,6.4,2.62,NaN,NaN
2742,"Zimbabwe,2000,Both sexes",21.6,12.9,3.44,46.57,41.03
2743,"Zimbabwe,2000,Male",21.6,20.6,3.75,45.15,40.39


#### 3. Combining airPollution DataFrames (3 - 7)

In [12]:
# For these DataFrames we only have information for 2016. 
combined_df = combined_df.merge(airPollutionDeathRate_pulmonary_df, how='outer', on='CASE_ID')
combined_df = combined_df.drop(combined_df.columns[[6, 7, 9, 10]], axis=1)
combined_df.rename(columns={'First Tooltip':'Air Pollution Death Rate per 100k (Pulmonary Disease)'}, inplace=True)

combined_df

,CASE_ID,30-70 Death (%),Crude Suicide Rate per 100k,Mortality by Poisoning Rate per 100k,Life Expectancy,Healthy Life Expectancy (HALE),Air Pollution Death Rate per 100k (Pulmonary Disease)
0,"Afghanistan,2016,Both sexes",29.8,0.0,1.19,NaN,NaN,6.97
1,"Afghanistan,2016,Male",31.8,0.0,0.53,NaN,NaN,6.79
2,"Afghanistan,2016,Female",27.7,0.0,1.89,NaN,NaN,7.17
3,"Afghanistan,2015,Both sexes",29.8,4.8,1.22,61.65,52.60,NaN
4,"Afghanistan,2015,Male",31.9,7.8,0.55,61.04,52.78,NaN
...,...,...,...,...,...,...,...
2740,"Zimbabwe,2005,Male",22.1,21.7,2.85,NaN,NaN,NaN
2741,"Zimbabwe,2005,Female",22.9,6.4,2.62,NaN,NaN,NaN
2742,"Zimbabwe,2000,Both sexes",21.6,12.9,3.44,46.57,41.03,NaN
2743,"Zimbabwe,2000,Male",21.6,20.6,3.75,45.15,40.39,NaN


In [13]:
# Repeat for DataFrames (4 - 7): Ischeamic heart disease, lower respiratory infections, strokes & cancers
combined_df = combined_df.merge(airPollutionDeathRate_ischeamic_df, how='outer', on='CASE_ID')
combined_df = combined_df.drop(combined_df.columns[[7, 8, 10, 11]], axis=1)
combined_df.rename(columns={'First Tooltip':'Air Pollution Death Rate per 100k (Ischeamic Heart Disease)'}, inplace=True)

combined_df = combined_df.merge(airPollutionDeathRate_respinfections_df, how='outer', on='CASE_ID')
combined_df = combined_df.drop(combined_df.columns[[8, 9, 11, 12]], axis=1)
combined_df.rename(columns={'First Tooltip':'Air Pollution Death Rate per 100k (Respiratory Infections)'}, inplace=True)

combined_df = combined_df.merge(airPollutionDeathRate_stroke_df, how='outer', on='CASE_ID')
combined_df = combined_df.drop(combined_df.columns[[9, 10, 12, 13]], axis=1)
combined_df.rename(columns={'First Tooltip':'Air Pollution Death Rate per 100k (Stroke)'}, inplace=True)

combined_df = combined_df.merge(airPollutionDeathRate_cancers_df, how='outer', on='CASE_ID')
combined_df = combined_df.drop(combined_df.columns[[10, 11, 13, 14]], axis=1)
combined_df.rename(columns={'First Tooltip':'Air Pollution Death Rate per 100k (Cancer)'}, inplace=True)

combined_df

,CASE_ID,30-70 Death (%),Crude Suicide Rate per 100k,Mortality by Poisoning Rate per 100k,Life Expectancy,Healthy Life Expectancy (HALE),Air Pollution Death Rate per 100k (Pulmonary Disease),Air Pollution Death Rate per 100k (Ischeamic Heart Disease),Air Pollution Death Rate per 100k (Respiratory Infections),Air Pollution Death Rate per 100k (Stroke),Air Pollution Death Rate per 100k (Cancer)
0,"Afghanistan,2016,Both sexes",29.8,0.0,1.19,NaN,NaN,6.97,38.51,31.26,15.75,1.61
1,"Afghanistan,2016,Male",31.8,0.0,0.53,NaN,NaN,6.79,42.23,29.37,14.02,2.24
2,"Afghanistan,2016,Female",27.7,0.0,1.89,NaN,NaN,7.17,34.56,33.27,17.60,0.95
3,"Afghanistan,2015,Both sexes",29.8,4.8,1.22,61.65,52.60,NaN,NaN,NaN,NaN,NaN
4,"Afghanistan,2015,Male",31.9,7.8,0.55,61.04,52.78,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2740,"Zimbabwe,2005,Male",22.1,21.7,2.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2741,"Zimbabwe,2005,Female",22.9,6.4,2.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2742,"Zimbabwe,2000,Both sexes",21.6,12.9,3.44,46.57,41.03,NaN,NaN,NaN,NaN,NaN
2743,"Zimbabwe,2000,Male",21.6,20.6,3.75,45.15,40.39,NaN,NaN,NaN,NaN,NaN


#### 4. Adding adolescentBirth DataFrame (2... and last one)

In [15]:
# For this particular DataFrame, we have information from years 2000-2017 but is not consistent across countries...
# e.g. Afghanistan has years 2000, -3, -5, -6, -7, -8, -9, -11, -13, -14, 17
# meanwhile Albania has complete info from 2000 to 2017. 

# Since we are working with years 2000, 2005, 2010, 2015 & 2016... only these years will be considered. 
adolescentBirth_df

,CASE_ID,Indicator_data,First Tooltip
0,"Afghanistan,2017",Adolescent birth rate (per 1000 women aged 15-...,62.0
1,"Afghanistan,2014",Adolescent birth rate (per 1000 women aged 15-...,77.2
2,"Afghanistan,2013",Adolescent birth rate (per 1000 women aged 15-...,87.0
3,"Afghanistan,2011",Adolescent birth rate (per 1000 women aged 15-...,125.7
4,"Afghanistan,2009",Adolescent birth rate (per 1000 women aged 15-...,80.0
...,...,...,...
2184,"Zimbabwe,2008",Adolescent birth rate (per 1000 women aged 15-...,112.0
2185,"Zimbabwe,2007",Adolescent birth rate (per 1000 women aged 15-...,102.6
2186,"Zimbabwe,2004",Adolescent birth rate (per 1000 women aged 15-...,98.7
2187,"Zimbabwe,2003",Adolescent birth rate (per 1000 women aged 15-...,101.0


In [16]:
# In order to merge the DataFrames correctly, we need to concatenate ",Both sexes" in the CASE_ID column
adolescentBirth_df['CASE_ID'] = adolescentBirth_df['CASE_ID'].astype(str) + ',Both sexes'
adolescentBirth_df

,CASE_ID,Indicator_data,First Tooltip
0,"Afghanistan,2017,Both sexes",Adolescent birth rate (per 1000 women aged 15-...,62.0
1,"Afghanistan,2014,Both sexes",Adolescent birth rate (per 1000 women aged 15-...,77.2
2,"Afghanistan,2013,Both sexes",Adolescent birth rate (per 1000 women aged 15-...,87.0
3,"Afghanistan,2011,Both sexes",Adolescent birth rate (per 1000 women aged 15-...,125.7
4,"Afghanistan,2009,Both sexes",Adolescent birth rate (per 1000 women aged 15-...,80.0
...,...,...,...
2184,"Zimbabwe,2008,Both sexes",Adolescent birth rate (per 1000 women aged 15-...,112.0
2185,"Zimbabwe,2007,Both sexes",Adolescent birth rate (per 1000 women aged 15-...,102.6
2186,"Zimbabwe,2004,Both sexes",Adolescent birth rate (per 1000 women aged 15-...,98.7
2187,"Zimbabwe,2003,Both sexes",Adolescent birth rate (per 1000 women aged 15-...,101.0


In [17]:
# Now we merge the DataFrames, only for years 2000, 2005, 2010, 2015 & 2016 (left join).
combined_df = combined_df.merge(adolescentBirth_df, how='left', on='CASE_ID')
combined_df = combined_df.drop(combined_df.columns[[11]], axis=1)
combined_df.rename(columns={'First Tooltip':'Adolescent Birth Rate per 1000 Women Aged 15-19'}, inplace=True)

combined_df.head(20)

,CASE_ID,30-70 Death (%),Crude Suicide Rate per 100k,Mortality by Poisoning Rate per 100k,Life Expectancy,Healthy Life Expectancy (HALE),Air Pollution Death Rate per 100k (Pulmonary Disease),Air Pollution Death Rate per 100k (Ischeamic Heart Disease),Air Pollution Death Rate per 100k (Respiratory Infections),Air Pollution Death Rate per 100k (Stroke),Air Pollution Death Rate per 100k (Cancer),Adolescent Birth Rate per 1000 Women Aged 15-19
0,"Afghanistan,2016,Both sexes",29.8,0.0,1.19,NaN,NaN,6.97,38.51,31.26,15.75,1.61,NaN
1,"Afghanistan,2016,Male",31.8,0.0,0.53,NaN,NaN,6.79,42.23,29.37,14.02,2.24,NaN
2,"Afghanistan,2016,Female",27.7,0.0,1.89,NaN,NaN,7.17,34.56,33.27,17.60,0.95,NaN
3,"Afghanistan,2015,Both sexes",29.8,4.8,1.22,61.65,52.60,NaN,NaN,NaN,NaN,NaN,NaN
4,"Afghanistan,2015,Male",31.9,7.8,0.55,61.04,52.78,NaN,NaN,NaN,NaN,NaN,NaN
5,"Afghanistan,2015,Female",27.8,1.5,1.92,62.35,52.44,NaN,NaN,NaN,NaN,NaN,NaN
6,"Afghanistan,2010,Both sexes",31.7,5.1,1.42,59.94,51.13,NaN,NaN,NaN,NaN,NaN,NaN
7,"Afghanistan,2010,Male",34.1,8.6,0.76,59.60,51.51,NaN,NaN,NaN,NaN,NaN,NaN
8,"Afghanistan,2010,Female",29.4,1.4,2.11,60.30,50.75,NaN,NaN,NaN,NaN,NaN,NaN
9,"Afghanistan,2005,Both sexes",34.1,6.3,1.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117.4


#### 5. Filling the NaN values and exporting to CSV file

In [33]:
# Replacing NaN's with zeroes.
combined_df.fillna(0, inplace=True)
combined_df

,CASE_ID,30-70 Death (%),Crude Suicide Rate per 100k,Mortality by Poisoning Rate per 100k,Life Expectancy,Healthy Life Expectancy (HALE),Air Pollution Death Rate per 100k (Pulmonary Disease),Air Pollution Death Rate per 100k (Ischeamic Heart Disease),Air Pollution Death Rate per 100k (Respiratory Infections),Air Pollution Death Rate per 100k (Stroke),Air Pollution Death Rate per 100k (Cancer),Adolescent Birth Rate per 1000 Women Aged 15-19
0,"Afghanistan,2016,Both sexes",29.8,0.0,1.19,0.00,0.00,6.97,38.51,31.26,15.75,1.61,0.0
1,"Afghanistan,2016,Male",31.8,0.0,0.53,0.00,0.00,6.79,42.23,29.37,14.02,2.24,0.0
2,"Afghanistan,2016,Female",27.7,0.0,1.89,0.00,0.00,7.17,34.56,33.27,17.60,0.95,0.0
3,"Afghanistan,2015,Both sexes",29.8,4.8,1.22,61.65,52.60,0.00,0.00,0.00,0.00,0.00,0.0
4,"Afghanistan,2015,Male",31.9,7.8,0.55,61.04,52.78,0.00,0.00,0.00,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2740,"Zimbabwe,2005,Male",22.1,21.7,2.85,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
2741,"Zimbabwe,2005,Female",22.9,6.4,2.62,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
2742,"Zimbabwe,2000,Both sexes",21.6,12.9,3.44,46.57,41.03,0.00,0.00,0.00,0.00,0.00,0.0
2743,"Zimbabwe,2000,Male",21.6,20.6,3.75,45.15,40.39,0.00,0.00,0.00,0.00,0.00,0.0


In [ ]:
# Writing the CSV file
combined_df.to_csv(file_name, encoding='utf-8')
